# TP 03 - Architecture Transformer Complète

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Assembler un Transformer Encoder complet

---

## Objectifs pédagogiques

À la fin de ce TP, vous serez capable de :
1. Implémenter le Positional Encoding
2. Comprendre les connexions résiduelles et Layer Normalization
3. Assembler un bloc Transformer complet
4. Empiler plusieurs blocs pour créer un encodeur

## 0. Installation et imports

In [ ]:
!pip install torch matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

## 1. Récapitulatif : Nos briques de base

In [ ]:
# Multi-Head Attention (du TP précédent)
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        self.W_o = nn.Linear(embed_dim, embed_dim)
    
    def split_heads(self, x):
        batch_size, seq_len, _ = x.shape
        x = x.view(batch_size, seq_len, self.num_heads, self.d_k)
        return x.transpose(1, 2)
    
    def concat_heads(self, x):
        batch_size, _, seq_len, _ = x.shape
        x = x.transpose(1, 2).contiguous()
        return x.view(batch_size, seq_len, self.embed_dim)
    
    def forward(self, x, mask=None):
        Q = self.split_heads(self.W_q(x))
        K = self.split_heads(self.W_k(x))
        V = self.split_heads(self.W_v(x))
        
        scores = Q @ K.transpose(-2, -1) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask, float('-inf'))
        
        attn_weights = F.softmax(scores, dim=-1)
        attn_output = attn_weights @ V
        
        concat_output = self.concat_heads(attn_output)
        output = self.W_o(concat_output)
        
        return output, attn_weights

print("MultiHeadAttention chargé !")

---

## 2. Positional Encoding

### Le problème

L'attention est **permutation-invariante** : elle ne sait pas que "Le chat mange" ≠ "mange chat Le".

On doit injecter l'information de **position** dans les embeddings.

### La solution : Sinusoïdes

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d}}\right)$$

Où :
- `pos` : position dans la séquence (0, 1, 2, ...)
- `i` : dimension (0, 1, 2, ..., embed_dim/2)
- `d` : embed_dim

In [ ]:
# ============================================
# EXERCICE 1 : Implémenter Positional Encoding
# ============================================

class PositionalEncoding(nn.Module):
    """
    Positional Encoding avec sinusoïdes.
    
    Args:
        embed_dim: Dimension des embeddings
        max_len: Longueur maximale des séquences
        dropout: Taux de dropout
    """
    
    def __init__(self, embed_dim, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        # Créer la matrice de positional encoding
        pe = torch.zeros(max_len, embed_dim)
        
        # Positions : [0, 1, 2, ..., max_len-1]
        position = torch.arange(0, max_len).unsqueeze(1).float()
        
        # Diviseur : 10000^(2i/d)
        # Astuce : utiliser exp(log) pour la stabilité numérique
        div_term = torch.exp(
            torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim)
        )
        
        # TODO: Remplir pe avec sin et cos
        # pe[:, 0::2] = sin(position * div_term)  # indices pairs
        # pe[:, 1::2] = cos(position * div_term)  # indices impairs
        
        pe[:, 0::2] = None  # À compléter
        pe[:, 1::2] = None  # À compléter
        
        # Ajouter dimension batch
        pe = pe.unsqueeze(0)  # (1, max_len, embed_dim)
        
        # Enregistrer comme buffer (pas un paramètre entraînable)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        """
        Args:
            x: Embeddings, shape (batch, seq_len, embed_dim)
        """
        seq_len = x.size(1)
        # Ajouter positional encoding (tronqué à la longueur de la séquence)
        x = x + self.pe[:, :seq_len, :]
        return self.dropout(x)

In [ ]:
# Test et visualisation
pe = PositionalEncoding(embed_dim=64, max_len=100, dropout=0.0)

# Visualiser le positional encoding
pe_matrix = pe.pe[0, :50, :].numpy()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap
im = axes[0].imshow(pe_matrix.T, cmap='RdBu', aspect='auto')
axes[0].set_xlabel("Position")
axes[0].set_ylabel("Dimension")
axes[0].set_title("Positional Encoding (Heatmap)")
plt.colorbar(im, ax=axes[0])

# Courbes pour quelques dimensions
for d in [0, 1, 10, 20, 30]:
    axes[1].plot(pe_matrix[:, d], label=f'dim {d}')
axes[1].set_xlabel("Position")
axes[1].set_ylabel("Valeur")
axes[1].set_title("Positional Encoding (Courbes)")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Observation** : Les basses dimensions (0, 1) oscillent rapidement, les hautes dimensions (30) oscillent lentement. Cela permet au modèle de distinguer les positions proches ET lointaines.

---

## 3. Feed-Forward Network

Après l'attention, chaque position passe par un réseau feed-forward identique.

$$\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2$$

La dimension cachée est typiquement 4× embed_dim.

In [ ]:
# ============================================
# EXERCICE 2 : Feed-Forward Network
# ============================================

class FeedForward(nn.Module):
    """
    Feed-Forward Network (2 couches linéaires avec ReLU).
    
    Args:
        embed_dim: Dimension d'entrée/sortie
        ff_dim: Dimension cachée (typiquement 4 * embed_dim)
        dropout: Taux de dropout
    """
    
    def __init__(self, embed_dim, ff_dim=None, dropout=0.1):
        super().__init__()
        
        if ff_dim is None:
            ff_dim = 4 * embed_dim
        
        # TODO: Créer le réseau
        # linear1: embed_dim -> ff_dim
        # activation: ReLU (ou GELU)
        # dropout
        # linear2: ff_dim -> embed_dim
        
        self.linear1 = None  # À compléter
        self.linear2 = None  # À compléter
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()  # ou nn.GELU()
    
    def forward(self, x):
        # TODO: Implémenter le forward
        # x -> linear1 -> activation -> dropout -> linear2
        return None

In [ ]:
# Test
ff = FeedForward(embed_dim=32, ff_dim=128)
x = torch.randn(2, 6, 32)
out = ff(x)
print(f"FFN output shape: {out.shape}")  # (2, 6, 32)

---

## 4. Connexions Résiduelles + Layer Normalization

### Connexions résiduelles

$$\text{output} = x + \text{SubLayer}(x)$$

Permet aux gradients de mieux circuler (comme dans ResNet).

### Layer Normalization

Normalise sur la dimension des features (pas sur le batch comme BatchNorm).

$$\text{LayerNorm}(x) = \gamma \cdot \frac{x - \mu}{\sigma + \epsilon} + \beta$$

### Architecture d'un bloc

```
Input
  ↓
  ├──────────────────┐
  ↓                  │ (residual)
LayerNorm            │
  ↓                  │
Multi-Head Attention │
  ↓                  │
Dropout              │
  ↓                  │
  + ←────────────────┘
  ↓
  ├──────────────────┐
  ↓                  │ (residual)
LayerNorm            │
  ↓                  │
Feed-Forward         │
  ↓                  │
Dropout              │
  ↓                  │
  + ←────────────────┘
  ↓
Output
```

In [ ]:
# Démonstration Layer Normalization
x = torch.randn(2, 4, 8)  # (batch, seq, features)

# Layer Norm normalise sur la dernière dimension (features)
ln = nn.LayerNorm(8)
x_norm = ln(x)

print(f"Avant LayerNorm:")
print(f"  Moyenne par position: {x[0, 0].mean():.4f}")
print(f"  Std par position: {x[0, 0].std():.4f}")

print(f"\nAprès LayerNorm:")
print(f"  Moyenne par position: {x_norm[0, 0].mean():.4f}")
print(f"  Std par position: {x_norm[0, 0].std():.4f}")

---

## 5. Bloc Transformer Encoder

### Exercice 3 : Assembler le bloc complet

In [ ]:
class TransformerEncoderBlock(nn.Module):
    """
    Un bloc de Transformer Encoder.
    
    Args:
        embed_dim: Dimension des embeddings
        num_heads: Nombre de têtes d'attention
        ff_dim: Dimension du feed-forward (défaut: 4 * embed_dim)
        dropout: Taux de dropout
    """
    
    def __init__(self, embed_dim, num_heads, ff_dim=None, dropout=0.1):
        super().__init__()
        
        if ff_dim is None:
            ff_dim = 4 * embed_dim
        
        # TODO: Créer les composants
        # 1. Multi-Head Attention
        self.attention = None  # MultiHeadAttention(embed_dim, num_heads)
        
        # 2. Feed-Forward
        self.feed_forward = None  # FeedForward(embed_dim, ff_dim, dropout)
        
        # 3. Layer Normalizations (2)
        self.norm1 = None  # nn.LayerNorm(embed_dim)
        self.norm2 = None  # nn.LayerNorm(embed_dim)
        
        # 4. Dropout (2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        """
        Args:
            x: Input, shape (batch, seq_len, embed_dim)
            mask: Masque optionnel
        
        Returns:
            output: (batch, seq_len, embed_dim)
        """
        # TODO: Implémenter le forward
        # Note: On utilise "Pre-LN" (LayerNorm avant l'opération)
        
        # Bloc 1: Attention avec résiduel
        # residual = x
        # x = self.norm1(x)
        # x, _ = self.attention(x, mask)
        # x = self.dropout1(x)
        # x = residual + x
        
        # Bloc 2: Feed-Forward avec résiduel
        # residual = x
        # x = self.norm2(x)
        # x = self.feed_forward(x)
        # x = self.dropout2(x)
        # x = residual + x
        
        return None  # À compléter

In [ ]:
# Test du bloc
block = TransformerEncoderBlock(embed_dim=32, num_heads=4, dropout=0.1)

x = torch.randn(2, 6, 32)
out = block(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {out.shape}")  # Doit être identique

---

## 6. Transformer Encoder Complet

On empile N blocs et on ajoute l'embedding + positional encoding.

In [ ]:
# ============================================
# EXERCICE 4 : Transformer Encoder complet
# ============================================

class TransformerEncoder(nn.Module):
    """
    Transformer Encoder complet.
    
    Args:
        vocab_size: Taille du vocabulaire
        embed_dim: Dimension des embeddings
        num_heads: Nombre de têtes d'attention
        num_layers: Nombre de blocs Transformer
        ff_dim: Dimension du feed-forward
        max_len: Longueur max des séquences
        dropout: Taux de dropout
    """
    
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers,
                 ff_dim=None, max_len=512, dropout=0.1):
        super().__init__()
        
        # TODO: Créer les composants
        
        # 1. Token Embedding
        self.token_embedding = None  # nn.Embedding(vocab_size, embed_dim)
        
        # 2. Positional Encoding
        self.positional_encoding = None  # PositionalEncoding(embed_dim, max_len, dropout)
        
        # 3. Liste de N blocs Transformer
        self.layers = nn.ModuleList([
            # TransformerEncoderBlock(embed_dim, num_heads, ff_dim, dropout)
            # for _ in range(num_layers)
        ])
        
        # 4. Layer Norm finale
        self.final_norm = None  # nn.LayerNorm(embed_dim)
        
        self.embed_dim = embed_dim
    
    def forward(self, x, mask=None):
        """
        Args:
            x: Token indices, shape (batch, seq_len)
            mask: Masque optionnel
        
        Returns:
            output: (batch, seq_len, embed_dim)
        """
        # TODO: Implémenter
        # 1. Token embedding (+ scaling par sqrt(embed_dim))
        # x = self.token_embedding(x) * math.sqrt(self.embed_dim)
        
        # 2. Ajouter positional encoding
        # x = self.positional_encoding(x)
        
        # 3. Passer par tous les blocs
        # for layer in self.layers:
        #     x = layer(x, mask)
        
        # 4. Normalisation finale
        # x = self.final_norm(x)
        
        return None  # À compléter

In [ ]:
# Test
encoder = TransformerEncoder(
    vocab_size=1000,
    embed_dim=64,
    num_heads=4,
    num_layers=3,
    dropout=0.1
)

# Input : indices de tokens
token_ids = torch.randint(0, 1000, (2, 10))  # batch=2, seq=10

output = encoder(token_ids)

print(f"Token IDs shape: {token_ids.shape}")
print(f"Encoder output shape: {output.shape}")  # (2, 10, 64)

# Compter les paramètres
num_params = sum(p.numel() for p in encoder.parameters())
print(f"\nNombre de paramètres: {num_params:,}")

---

## 7. Application : Classification de séquences

Ajoutons une tête de classification pour faire de la classification de texte.

In [ ]:
class TransformerClassifier(nn.Module):
    """
    Transformer pour la classification de texte.
    Utilise le token [CLS] (premier token) pour la classification.
    """
    
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers,
                 num_classes, ff_dim=None, max_len=512, dropout=0.1):
        super().__init__()
        
        self.encoder = TransformerEncoder(
            vocab_size, embed_dim, num_heads, num_layers,
            ff_dim, max_len, dropout
        )
        
        # Tête de classification
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.Tanh(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim, num_classes)
        )
    
    def forward(self, x):
        # Encoder
        encoded = self.encoder(x)  # (batch, seq_len, embed_dim)
        
        # Prendre le premier token (équivalent [CLS])
        cls_output = encoded[:, 0, :]  # (batch, embed_dim)
        
        # Classifier
        logits = self.classifier(cls_output)  # (batch, num_classes)
        
        return logits

In [ ]:
# Test du classifieur
classifier = TransformerClassifier(
    vocab_size=1000,
    embed_dim=64,
    num_heads=4,
    num_layers=2,
    num_classes=3,  # ex: positif, négatif, neutre
    dropout=0.1
)

token_ids = torch.randint(0, 1000, (4, 20))  # 4 séquences de 20 tokens
logits = classifier(token_ids)

print(f"Input shape: {token_ids.shape}")
print(f"Logits shape: {logits.shape}")  # (4, 3)
print(f"\nPrédictions (après softmax): {F.softmax(logits, dim=-1)[0]}")

---

## 8. Récapitulatif

### Architecture Transformer Encoder

```
Tokens (indices)
      ↓
Token Embedding (× √d)
      ↓
Positional Encoding (+)
      ↓
┌─────────────────────┐
│  Transformer Block  │ × N
│  ├─ LayerNorm       │
│  ├─ Multi-Head Attn │
│  ├─ Residual        │
│  ├─ LayerNorm       │
│  ├─ Feed-Forward    │
│  └─ Residual        │
└─────────────────────┘
      ↓
Final LayerNorm
      ↓
Output (contextualized embeddings)
```

### Composants clés

| Composant | Rôle |
|-----------|------|
| Positional Encoding | Injecter l'information de position |
| Multi-Head Attention | Capturer les relations entre tokens |
| Feed-Forward | Transformation non-linéaire |
| Residuals | Faciliter le flux des gradients |
| Layer Norm | Stabiliser l'entraînement |

### Prochaine session

Nous allons **entraîner** ce Transformer sur une tâche de classification de texte (Fake News) !

---

## 9. Pour aller plus loin (optionnel)

### Visualisation des embeddings positionnels

In [ ]:
# Installation sklearn pour cette section optionnelle
!pip install scikit-learn -q

# Les positions proches ont des encodages similaires
pe = PositionalEncoding(embed_dim=64, max_len=100, dropout=0.0)
pe_matrix = pe.pe[0].numpy()

# Calculer la similarité cosinus entre positions
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(pe_matrix[:30])

plt.figure(figsize=(8, 6))
plt.imshow(similarity, cmap='RdBu')
plt.colorbar(label='Similarité cosinus')
plt.xlabel('Position')
plt.ylabel('Position')
plt.title('Similarité entre positions (Positional Encoding)')
plt.show()

print("Observation: Les positions proches ont des encodages plus similaires.")